In [27]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from IPython.display import clear_output

In [28]:
a = torch.rand([32,512,14,14])
a =nn.Conv2d(512,512,5,3)(a)
print(a.shape)

torch.Size([32, 512, 4, 4])


In [29]:
torch.cuda.is_available()

True

In [30]:
torch.cuda.current_device()

0

In [31]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)).cuda()])
trainset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data',train=True,download=False,transform=transform)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=64,shuffle=True,num_workers=2)
testset = torchvision.datasets.CIFAR100(root='C:\\Users\\get2b\\Desktop\\Arav\\AI ML\\DATA SETS\\pytorch Data',train=False,download=False,transform=transform)
test_loader = torch.utils.data.DataLoader(testset,batch_size=1000,shuffle=True,num_workers=2)

In [32]:
class inception_module(nn.Module):
    
    def __init__(self,input_size,f1x1,fd3x3,f3x3,fd5x5,f5x5,poolProj):
        
        super().__init__()
        
        self.incept1 = nn.Sequential(nn.Conv2d(input_size,f1x1,1,1,0),
                                     nn.BatchNorm2d(f1x1,eps=0.001),
                                     nn.ReLU(inplace=True))
        self.incept2 = nn.Sequential(nn.Conv2d(input_size,fd3x3,1,1,0),
                                     nn.BatchNorm2d(fd3x3,eps=0.001),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(fd3x3,f3x3,3,1,1),
                                     nn.BatchNorm2d(f3x3,eps=0.001),
                                     nn.ReLU(inplace=True))
        self.incept3 = nn.Sequential(nn.Conv2d(input_size,fd5x5,1,1,0),
                                     nn.BatchNorm2d(fd5x5,eps=0.001),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(fd5x5,f5x5,5,1,2),
                                     nn.BatchNorm2d(f5x5,eps=0.001),
                                     nn.ReLU(inplace=True))
        self.incept4 = nn.Sequential(nn.MaxPool2d(3,1,1),
                                     nn.ReLU(inplace=True),
                                     nn.Conv2d(input_size,poolProj,1,1,0),
                                     nn.BatchNorm2d(poolProj,eps=0.001),
                                     nn.ReLU(inplace=True))
        
    def forward(self,x):
        
        x1 = self.incept1(x)
        x2 = self.incept2(x)
        x3 = self.incept3(x)
        x4 = self.incept4(x)
        xcat = torch.cat([x1,x2,x3,x4],dim=1)
        
        return xcat
        

In [33]:
class aux_output(nn.Module):
    
    def __init__(self,input_size):
    
        super().__init__()
        # Have to change this too just for the sake of running it on CIFAR 100
        self.aux = nn.Sequential(nn.Conv2d(input_size,input_size,2),# For imagenet dataset itll be (input_size,input_size,5,3)
                                 nn.BatchNorm2d(input_size,eps=0.001),
                                 nn.ReLU(inplace=True),
                                 nn.Conv2d(input_size,128,1,1),
                                 nn.BatchNorm2d(128,eps=0.001),
                                 nn.ReLU(inplace=True),
                                 nn.Flatten(),
                                 nn.Linear(128,1024),               # For imagenet dataset nn.Linear(2048,1024)
                                 nn.ReLU(inplace=True),
                                 nn.Dropout(p=0.7),
                                 nn.Linear(1024,100),              # For imagenet dataset nn.Linear(1024,1000)
                                 nn.ReLU(inplace=True))
        #Self.auxp was just made for the purpose of CIFAR 100 Dataset and nothing else
        self.auxp = nn.Sequential(nn.Conv2d(input_size,128,1,1),
                                 nn.BatchNorm2d(128,eps=0.001),
                                 nn.ReLU(inplace=True),
                                 nn.Flatten(),
                                 nn.Linear(512,1024),               # For imagenet dataset nn.Linear(2048,1024)
                                 nn.ReLU(inplace=True),
                                 nn.Dropout(p=0.7),
                                 nn.Linear(1024,100),              # For imagenet dataset nn.Linear(1024,1000)
                                 nn.ReLU(inplace=True))
        
    def forward(self,x):
        
        #x = self.aux(x) This is for imagenet dataset
        x = self.auxp(x)
        
        return x        

In [34]:
class GoogLeNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self.top = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=64,kernel_size=7,stride=2, padding=3),
                                 nn.BatchNorm2d(64,eps=0.001),
                                 nn.ReLU(inplace=True),
                                 nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
                                 nn.LocalResponseNorm(1),
                                 nn.Conv2d(64,192,3,1,1),
                                 nn.BatchNorm2d(192,eps=0.001),
                                 nn.ReLU(inplace=True),
                                 nn.MaxPool2d(3,2,1))
        self.inception3a = inception_module(192,64,96,128,16,32,32)
        self.inception3b = inception_module(256,128,128,192,32,96,64)
        self.maxp = nn.MaxPool2d(3,2,1)
        self.inception4a = inception_module(480,192,96,208,16,48,64)
        self.inception4b = inception_module(512,160,112,224,24,64,64)
        self.inception4c = inception_module(512,128,128,256,24,64,64)
        self.inception4d = inception_module(512,112,144,288,32,64,64)
        self.inception4e = inception_module(528,256,160,320,32,128,128)
        self.inception5a = inception_module(832,256,160,320,32,128,128)
        self.inception5b = inception_module(832,384,192,384,48,128,128)
        self.avgp = nn.AvgPool2d(2)              # for 3,224,224 shaped input avgpool needs to be 7
        self.linear = nn.Linear(832,100)       # for 3,224,224 shaped input linear should be 1024,1000
        self.aux1 = aux_output(input_size=512)
        self.aux2 = aux_output(input_size=528)
        
    def forward(self,x):
        
        x = self.top(x)
        
        x = self.inception3a(x)
        x = self.inception3b(x)
        
        x = self.maxp(x)
        
        x = self.inception4a(x)
        if self.training:    
            aux1 = self.aux1(x)  
        
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        if self.training: 
            aux2 = self.aux2(x)  
        
        x = self.inception4e(x)
        
        x = self.maxp(x)
        
        #x = self.inception5a(x)
        #x = self.inception5b(x)
        #x = self.avgp(x) 
        x = nn.Flatten()(x)
        x = nn.Dropout(p=0.4)(x)
        x = self.linear(x)
        x = nn.ReLU()(x)
        
        #return [out,aux1,aux2]
        if self.training: 
            return [x,aux1,aux2]
        return x

In [35]:
model = GoogLeNet().cuda()
#model.load_state_dict(torch.load('C:\\Users\\get2b\\Desktop\\Arav\\Neural Networks\\Pytorch implementation\\All Models Saved\\GoogLeNet.pth'))
loss_function1 = nn.CrossEntropyLoss()
loss_function2 = nn.CrossEntropyLoss()
loss_function3 = nn.CrossEntropyLoss()
optimizer0 = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer1 = torch.optim.SGD(aux_output(input_size=512).parameters(), lr=0.001, momentum=0.9) 
optimizer2 = torch.optim.SGD(aux_output(input_size=528).parameters(), lr=0.001, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer0,
                                                 base_lr=1e-4,
                                                 max_lr=1e-3,
                                                 step_size_up=5,
                                                 step_size_down=5,
                                                 mode="exp_range",
                                                 cycle_momentum=False)

In [36]:
model.parameters

<bound method Module.parameters of GoogLeNet(
  (top): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): LocalResponseNorm(1, alpha=0.0001, beta=0.75, k=1.0)
    (5): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception3a): inception_module(
    (incept1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (incept2): Sequential(
      (0): Conv2d(192, 96, kernel_si

In [37]:
def one_hot_encoder(labels):
    final_to_return = []
    labels.numpy()
    for i in range(len(labels)):
        oneHot = [0]*100
        oneHot[labels[i]] = 1
        final_to_return.append(oneHot)
    return torch.Tensor(final_to_return)

In [38]:
def test():
    model.eval()
    total = 0.0
    accuracy = 0.0
    with torch.no_grad():
        
        for i ,data in enumerate(test_loader,0):
            
            inputs, labels = data
            inputs, labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total+=labels.size(0)
            accuracy+=(predicted == labels).sum().item()
    
    print("{}/{}".format(accuracy,total))
    accuracy = float(100 * accuracy / total)
    
    return(accuracy)



def train(epoch):
    for ep in range(1,epoch+1):
        print("Epoch : ",ep)
        running_loss=0
        flag = True
        model.train()
        for i, data in enumerate(train_loader,0):
            inputs,labels = data
            labels = one_hot_encoder(labels)
            inputs,labels = inputs.cuda(),labels.cuda()
            optimizer0.zero_grad(set_to_none=False)
            optimizer1.zero_grad(set_to_none=False)
            optimizer2.zero_grad(set_to_none=False)
            outputs = model(inputs)
            lossPrimary = loss_function1(outputs[0],labels.squeeze(1))
            loss2 = loss_function2(outputs[1],labels)
            loss3 = loss_function3(outputs[2],labels)
            torch.autograd.backward([lossPrimary,loss2,loss3])
            optimizer0.step()
            optimizer1.step()
            optimizer2.step()
            running_loss+=lossPrimary.item()
            if (i+1)%1000 == 0 :
                print("[{}/{}]".format(i+1,len(train_loader.dataset)))
                print("Loss of the training model is : %.4f"%(running_loss/1000))
                running_loss=0
        lr_scheduler.step()
        accuracy = test()
        print('For epoch', ep,'the test accuracy over the whole test set is %.2f %%' % (accuracy))

In [39]:
train(20)

Epoch :  1
452.0/10000.0
For epoch 1 the test accuracy over the whole test set is 4.52 %
Epoch :  2
971.0/10000.0
For epoch 2 the test accuracy over the whole test set is 9.71 %
Epoch :  3
1556.0/10000.0
For epoch 3 the test accuracy over the whole test set is 15.56 %
Epoch :  4
1961.0/10000.0
For epoch 4 the test accuracy over the whole test set is 19.61 %
Epoch :  5
2575.0/10000.0
For epoch 5 the test accuracy over the whole test set is 25.75 %
Epoch :  6
2852.0/10000.0
For epoch 6 the test accuracy over the whole test set is 28.52 %
Epoch :  7
3398.0/10000.0
For epoch 7 the test accuracy over the whole test set is 33.98 %
Epoch :  8
3770.0/10000.0
For epoch 8 the test accuracy over the whole test set is 37.70 %
Epoch :  9
3991.0/10000.0
For epoch 9 the test accuracy over the whole test set is 39.91 %
Epoch :  10
4143.0/10000.0
For epoch 10 the test accuracy over the whole test set is 41.43 %
Epoch :  11
4296.0/10000.0
For epoch 11 the test accuracy over the whole test set is 42.96 %

In [41]:
torch.save(model.state_dict(), 'C:\\Users\\get2b\\Desktop\\Arav\\Neural Networks\\Pytorch implementation\\All Models Saved\\GoogLeNet.pth')